In [2]:
%pip install --upgrade pip
%pip install ydf
%pip install tf_keras
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import ydf
import pandas as pd
import numpy as np
import tensorflow as tf
import tf_keras
import math

In [9]:
dataset_df = pd.read_csv("data/cvd_v2.csv", sep=',')


FileNotFoundError: [Errno 2] No such file or directory: 'data/cvd_v2.csv'

In [ ]:
# 将数据集分割为训练集和测试集。

def split_dataset(dataset, test_ratio=0.005):
  """将panda数据框分割成两部分。"""
  # 生成一个与数据集长度相同的随机数组，元素值小于测试比例的为True，大于等于测试比例的为False
  test_indices = np.random.rand(len(dataset)) < test_ratio
  # 返回测试集和训练集
  return dataset[~test_indices], dataset[test_indices]


# 调用split_dataset函数将数据集分割成训练集和测试集，并将返回的结果分别赋值给train_ds_pd和test_ds_pd
train_ds_pd, test_ds_pd = split_dataset(dataset_df)

# 打印训练集和测试集的样本数量
print("{}个样本用于训练，{}个样本用于测试。".format(
    len(train_ds_pd), len(test_ds_pd)))


68335个样本用于训练，350个样本用于测试。


In [ ]:
tuner = ydf.RandomSearchTuner(num_trials=10, automatic_search_space=True)
# model = ydf.RandomForestLearner(label="cardio",num_trees=270).train(train_ds_pd)
# evaluation = model.evaluate(test_ds_pd)
model_2 = ydf.GradientBoostedTreesLearner(label="cardio",tuner=tuner,num_trees=50).train(train_ds_pd)
evaluation_2 = model_2.evaluate(test_ds_pd)
# model_3 = ydf.RandomForestLearner(
#     label="cardio",
#     # Tell the Random Forest to compute Out-of-bag variable importances.
#     # This is expensive but gives the best results.
#     compute_oob_variable_importances=True,
#     # Enable feature selection using the "backward selection" algorithm.
#     feature_selector=ydf.BackwardSelectionFeatureSelector(),
# ).train(train_ds_pd)

# evaluation_3 = model_3.evaluate(test_ds_pd)
# print(evaluation)
print(evaluation_2)
# print(evaluation_3)


Train model on 68335 examples


ValueError: INVALID_ARGUMENT: Operation interrupted by user

In [ ]:

model.describe()